# - Create a **Comprehensive Report** and **Dashboard** to present the insights from our Hotel Booking Analysis. We'll use a combination of Python libraries to create visualizations and a dashboard-style report

In [3]:
#import libraries ----
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
#import the libraries for plotting
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import display, Markdown, HTML


In [5]:

# Assuming we've already done the data preprocessing and analysis as in the previous script
# Load the preprocessed data
df = pd.read_excel('hotel_bookings.xlsx')


In [6]:
# Create a Jupyter Notebook cell for the title
display(Markdown("# Hotel Booking Analysis Dashboard"))

# 1. Overview Statistics
display(Markdown("## 1. Overview Statistics"))

total_bookings = len(df)
cancellation_rate = df['is_canceled'].mean()
avg_daily_rate = df['adr'].mean()
avg_lead_time = df['lead_time'].mean()

overview_stats = pd.DataFrame({
    'Metric': ['Total Bookings', 'Cancellation Rate', 'Average Daily Rate', 'Average Lead Time'],
    'Value': [total_bookings, f"{cancellation_rate:.2%}", f"${avg_daily_rate:.2f}", f"{avg_lead_time:.1f} days"]
})
# Style the DataFrame
styled_df = overview_stats.style.set_properties(**{'text-align': 'left'})
styled_df = styled_df.set_table_styles([
    {'selector': 'th', 'props': [('background-color', 'paleturquoise'), ('color', 'black')]},
    {'selector': 'td', 'props': [('background-color', 'lavender')]}
])

# Display the styled DataFrame
display(HTML(styled_df.to_html(index=False)))

# Hotel Booking Analysis Dashboard

## 1. Overview Statistics

,Metric,Value
0,Total Bookings,119390
1,Cancellation Rate,37.04%
2,Average Daily Rate,$101.83
3,Average Lead Time,104.0 days


In [7]:
# 2. Booking Trends
display(Markdown("## 2. Booking Trends"))

# Monthly bookings
df['arrival_date'] = pd.to_datetime(df['arrival_date_year'].astype(str) + '-' + df['arrival_date_month'].astype(str) + '-' + df['arrival_date_day_of_month'].astype(str))
monthly_bookings = df.groupby(df['arrival_date'].dt.to_period('M')).size().reset_index(name='count')
monthly_bookings['arrival_date'] = monthly_bookings['arrival_date'].dt.to_timestamp()

fig = px.line(monthly_bookings, x='arrival_date', y='count', title='Monthly Booking Trends')
fig.show()

## 2. Booking Trends

In [8]:
# 3. Hotel Type Comparison
df['stays_total_nights'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']
display(Markdown("## 3. Hotel Type Comparison"))

hotel_comparison = df.groupby('hotel').agg({
    'is_canceled': 'mean',
    'adr': 'mean',
    'total_of_special_requests': 'mean',
    'stays_total_nights': 'mean'
}).reset_index()

fig = make_subplots(rows=2, cols=2, subplot_titles=("Cancellation Rate", "Average Daily Rate",
                                                    "Avg. Special Requests", "Avg. Length of Stay"))

fig.add_trace(go.Bar(x=hotel_comparison['hotel'], y=hotel_comparison['is_canceled'], name="Cancellation Rate"), row=1, col=1)
fig.add_trace(go.Bar(x=hotel_comparison['hotel'], y=hotel_comparison['adr'], name="ADR"), row=1, col=2)
fig.add_trace(go.Bar(x=hotel_comparison['hotel'], y=hotel_comparison['total_of_special_requests'], name="Special Requests"), row=2, col=1)
fig.add_trace(go.Bar(x=hotel_comparison['hotel'], y=hotel_comparison['stays_total_nights'], name="Length of Stay"), row=2, col=2)

fig.update_layout(height=800, width=800, title_text="Hotel Type Comparison")
fig.show()

## 3. Hotel Type Comparison

In [9]:
# 4. Booking Channels
display(Markdown("## 4. Booking Channels"))

booking_channels = df['market_segment'].value_counts().reset_index()
booking_channels.columns = ['channel', 'count']

fig = px.pie(booking_channels, values='count', names='channel', title='Booking Channels')
fig.show()

## 4. Booking Channels

In [10]:
# 5. ADR by Length of Stay
display(Markdown("## 5. ADR by Length of Stay"))

stay_adr = df.groupby('stays_total_nights')['adr'].mean().reset_index()

fig = px.line(stay_adr, x='stays_total_nights', y='adr', title='Average Daily Rate by Length of Stay')
fig.update_layout(xaxis_title="Number of Nights", yaxis_title="Average Daily Rate")
fig.show()

## 5. ADR by Length of Stay

In [11]:
# 6. Special Requests Analysis
display(Markdown("## 6. Special Requests Analysis"))

special_requests = df['total_of_special_requests'].value_counts().sort_index().reset_index()
special_requests.columns = ['number_of_requests', 'count']

fig = px.bar(special_requests, x='number_of_requests', y='count', title='Distribution of Special Requests')
fig.update_layout(xaxis_title="Number of Special Requests", yaxis_title="Count of Bookings")
fig.show()

## 6. Special Requests Analysis

In [12]:
# 7. Lead Time Analysis
display(Markdown("## 7. Lead Time Analysis"))

df['lead_time_category'] = pd.cut(df['lead_time'], bins=[0, 7, 30, 90, 180, np.inf], labels=['0-7 days', '8-30 days', '31-90 days', '91-180 days', '180+ days'])
lead_time_adr = df.groupby('lead_time_category')['adr'].mean().reset_index()

fig = px.bar(lead_time_adr, x='lead_time_category', y='adr', title='Average Daily Rate by Lead Time')
fig.update_layout(xaxis_title="Lead Time", yaxis_title="Average Daily Rate")
fig.show()

## 7. Lead Time Analysis

In [13]:
# 8. Cancellation Analysis
display(Markdown("## 8. Cancellation Analysis"))

cancellation_lead_time = df.groupby('is_canceled')['lead_time'].mean().reset_index()
cancellation_lead_time['is_canceled'] = cancellation_lead_time['is_canceled'].map({0: 'Not Canceled', 1: 'Canceled'})

fig = px.bar(cancellation_lead_time, x='is_canceled', y='lead_time', title='Average Lead Time for Canceled vs Non-Canceled Bookings')
fig.update_layout(xaxis_title="Booking Status", yaxis_title="Average Lead Time (days)")
fig.show()

## 8. Cancellation Analysis

In [14]:

# 9. Key Insights and Recommendations
display(Markdown("## 9. Key Insights and Recommendations"))

insights = [
    "The overall cancellation rate is {:.2%}. Implementing a more flexible cancellation policy might help reduce this.".format(cancellation_rate),
    "City hotels have a higher average daily rate (${:.2f}) compared to resort hotels (${:.2f}). This suggests potential for price optimization in resort hotels.".format(
        df[df['hotel'] == 'City Hotel']['adr'].mean(),
        df[df['hotel'] == 'Resort Hotel']['adr'].mean()
    ),
    "Bookings with longer lead times tend to have lower average daily rates. Consider implementing dynamic pricing strategies based on lead time.",
    "The most common number of special requests is {}. Staff should be prepared to handle this number of requests efficiently.".format(
        df['total_of_special_requests'].mode().values[0]
    ),
    "The average length of stay is {:.1f} nights. Offering package deals or discounts for longer stays might increase this.".format(
        df['stays_total_nights'].mean()
    )
]

for i, insight in enumerate(insights, 1):
    display(Markdown(f"{i}. {insight}"))


## 9. Key Insights and Recommendations

1. The overall cancellation rate is 37.04%. Implementing a more flexible cancellation policy might help reduce this.

2. City hotels have a higher average daily rate ($105.30) compared to resort hotels ($94.95). This suggests potential for price optimization in resort hotels.

3. Bookings with longer lead times tend to have lower average daily rates. Consider implementing dynamic pricing strategies based on lead time.

4. The most common number of special requests is 0. Staff should be prepared to handle this number of requests efficiently.

5. The average length of stay is 3.4 nights. Offering package deals or discounts for longer stays might increase this.

In [15]:
# 10. Conclusion
display(Markdown("## 10. Conclusion"))

display(Markdown("""
This analysis provides valuable insights into booking patterns, pricing strategies, and customer behavior for our hotels.
Key areas for focus include:
- Optimizing pricing strategies based on lead time and length of stay
- Improving cancellation policies to reduce the cancellation rate
- Enhancing services to accommodate common special requests
- Developing targeted marketing strategies for different types of customers and booking channels

Continuous monitoring and analysis of these metrics will be crucial for ongoing improvement in hotel performance and customer satisfaction.
"""))

## 10. Conclusion


This analysis provides valuable insights into booking patterns, pricing strategies, and customer behavior for our hotels.
Key areas for focus include:
- Optimizing pricing strategies based on lead time and length of stay
- Improving cancellation policies to reduce the cancellation rate
- Enhancing services to accommodate common special requests
- Developing targeted marketing strategies for different types of customers and booking channels

Continuous monitoring and analysis of these metrics will be crucial for ongoing improvement in hotel performance and customer satisfaction.
